In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../../')

In [39]:
from datetime import datetime
from glob import glob
import os
import numpy as np
import pandas as pd

from tqdm import tqdm

from utils.db_util import create_postgres_engine, make_query


In [4]:
INPUT_DIR = '/home/sjb/Projects/Research/LinkedIn_OB/data/company_level_individual_profiles'
OUTPUT_DIR = '/home/sjb/Projects/Research/LinkedIn_OB/data/company_level_individual_stay_term'

In [5]:
db_conn = create_postgres_engine()

In [6]:
source_files = glob(os.path.join(INPUT_DIR,
                                 '*.csv'))

In [7]:
sample = pd.read_csv(source_files[0])

In [12]:
org_profile_link = sample['org_profile_link'].values[0]
person_id_tuple = tuple(sample['person_id'].tolist())

In [15]:
q = """
    SELECT *
    FROM linkedin.person_experience
    WHERE org_profile_link = '{org_profile_link}'
    AND person_id IN {person_ids}
""".format(org_profile_link=org_profile_link,
           person_ids=person_id_tuple)

In [16]:
org_person_df = make_query(q, db_conn)

In [20]:
assert len(set(org_person_df['person_id']).symmetric_difference(set(person_id_tuple))) == 0

In [21]:
len(person_id_tuple), org_person_df.shape

(2712, (3956, 12))

In [30]:
MONTH_MAP = {
    'January': 1,
    'February': 2,
    'March': 3,
    'April': 4,
    'May': 5,
    'June': 6,
    'July': 7,
    'August': 8,
    'September': 9,
    'October': 10,
    'November': 11,
    'December': 12,
}

_MIN_WORK_YEAR = 1940
_MAX_WORK_YEAR = 2014

_PRESENT_Y_M = '2013-11'

In [45]:
def _validate_month(month_string):
    if month_string not in MONTH_MAP:
        raise ValueError('month string {} is  not a valid month!'.format(month_string))

def _validate_year(year_string):
    if not year_string.isnumeric():
        raise ValueError('expected year string {} to be of numbers'.format(year_string))
    
    year_num = int(year_string)
    if (year_num > _MAX_WORK_YEAR) or (year_num < _MIN_WORK_YEAR):
        raise ValueError('year is {} which is outside the allowable range'.format(year_num))
        
def parse_date(date_string, date_type='start_date'):
    try:
        if date_string == 'Present':
            return datetime.strptime(_PRESENT_Y_M, '%Y-%m')

        splits = date_string.split()

        if len(splits) == 1:
            year = splits[0]
            # assume January start, December end if only yr provided
            if date_type == 'start_date':
                month_number = 1
            elif date_type == 'end_date':
                month_number = 12
            else:
                raise ValueError('invalid date_type arg {}'.format(date_type))
        elif len(splits) == 2:
            month, year = splits
            _validate_month(month)
            month_number = MONTH_MAP[month]
        else:
            raise ValueError('Unexpected date string format with split length {}'.format(len(splits)))

        _validate_year(year)

        y_m_string = '{}-{}'.format(year, month_number)
        y_m = datetime.strptime(y_m_string, '%Y-%m')

        return y_m
    
    except Exception:
        return np.nan

def parse_duration(duration_string):
    splits = duration_string.split()
    l = len(splits)
    
    if l not in [2, 4]:
        raise ValueError('Unexpected duration string format of length {}'.format(l))
        
    num_months = 0
    
    for i in range(1, l, 2):
        if splits[i] == 'years':
            num_months += int(splits[i-1]) * 12
        elif splits[i] in ['month', 'months']:
            num_months += int(splits[i-1])
        else:
            raise ValueError('Unexpected duration string format showing {}'.format(splits[i]))
    
    return num_months
    

def diff_month(date_end, date_start):
    n_months = (date_end.year - date_start.year) * 12 + date_end.month - date_start.month + 1
    return n_months

In [46]:
org_person_df['date_start_parsed'] = org_person_df['date_start'].apply(lambda x: parse_date(x, 
                                                                       date_type='start_date'))

org_person_df['date_end_parsed'] = org_person_df['date_end'].apply(lambda x: parse_date(x, 
                                                                   date_type='end_date'))

In [51]:
org_person_df.head()

,person_id,experience_id,org_name,org_profile_link,org_detail,experience_title,experience_location,experience_description,date_start,date_end,duration,is_current,date_start_parsed,date_end_parsed
0,101298151_2026_2000,0,Iowa State University,/company/iowa-state-university?trk=ppro_cprof,"Educational Institution; 5001-10,000 employees...",Graduate Research Assistant,None,"Department of Kinesiology, ISU\n• Anti-viral a...",August 2008,August 2013,5 years 1 month,False,2008-08-01,2013-08-01
1,101298151_2026_2000,1,Iowa State University,/company/iowa-state-university?trk=ppro_cprof,"Educational Institution; 5001-10,000 employees...",PhD Candidate,"Ames, Iowa",Worked under the supervision of Dr. Marian Koh...,August 2008,August 2013,5 years 1 month,False,2008-08-01,2013-08-01
2,101299072_2026_2000,0,"Iowa State University, School of Education",/company/iowa-state-university?trk=ppro_cprof,"Educational Institution; 5001-10,000 employees...",Graphic Designer,None,• Design webpage templates and layouts for fac...,September 2010,Present,3 years 3 months,True,2010-09-01,2013-11-01
3,101299072_2026_2000,1,"Iowa State University, Office of Community Col...",/company/iowa-state-university?trk=ppro_cprof,"Educational Institution; 5001-10,000 employees...",Undergraduate Research Associate / Graphic Des...,None,• Designed webpage templates and layouts for f...,September 2008,August 2010,2 years,False,2008-09-01,2010-08-01
4,101308659_2027_2000,3,Iowa State University,/company/iowa-state-university?trk=ppro_cprof,"Educational Institution; 5001-10,000 employees...",Assistant Professor,None,None,August 1988,May 1999,10 years 10 months,False,1988-08-01,1999-05-01


In [ ]:
a = temp[temp['person_id'] == '62398797_1248_1000_1500'].copy()

In [ ]:
a = a[['date_start', 'date_end', 'duration']].copy()

In [ ]:
a['duration_parsed'] = a['duration'].apply(parse_duration)

In [ ]:
a['date_start_parsed'] = a['date_start'].apply(lambda x: parse_date(x, 
                                                                    date_type='start_date'))

a['date_end_parsed'] = a['date_end'].apply(lambda x: parse_date(x, 
                                                                date_type='end_date'))

In [ ]:
a['calc_duration'] = a.apply(lambda r: diff_month(r['date_end_parsed'],
                                                  r['date_start_parsed']), axis=1)

In [ ]:
a

In [ ]:
temp['person_id'].value_counts()

In [ ]:
sample.shape

In [43]:
sample.loc[sample['person_id'] == '101915422_2039_2000']

,org_profile_link,person_id,parent_folder,batch_folder,given_name,family_name,header_title,header_location,header_industry,country,person_summary,word_length,char_length
37,/company/iowa-state-university?trk=ppro_cprof,101915422_2039_2000,2039,2000,Andrei,Migunov,Research Assistant at the Laboratory for Algor...,"Ames, Iowa",Computer Software,US,I am currently an undergraduate at Iowa State ...,156,735


In [ ]:
sample.loc[sample['person_id'] == '62398797_1248_1000_1500', 'person_summary'].values[0]